In [641]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [642]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def line_slope(x1, y1, x2, y2):
    return ((y2 - y1) / (x2 - x1))

def is_left ( x1, y1, x2, y2 ):
    return line_slope( x1, y1, x2, y2 ) < 0

def bad_slope ( x1, y1, x2, y2 ):
    slope = line_slope(x1, y1, x2, y2)
    if ( slope < 0 ):
        return slope > -.5 or slope < -.8
    else:
        return slope < .5 or slope > .8
        
def append_coordinates ( x1, y1, x2, y2, coords ) :
    coords['x1'].append(x1)
    coords['y1'].append(y1)
    coords['x2'].append(x2)
    coords['y2'].append(y2)
    
def render_coordinates( img, coords, top, bottom, color, thickness ) :
    try:
        avg_x1 = int(np.mean(coords['x1']))
        avg_y1 = int(np.mean(coords['y1']))
        avg_x2 = int(np.mean(coords['x2']))
        avg_y2 = int(np.mean(coords['y2']))

        slope = line_slope(avg_x1, avg_y1, avg_x2, avg_y2)

        if( slope != 0 ):
            y1 = top
            x1 = int(avg_x1 + ( y1 - avg_y1) / slope)
            y2 = bottom
            x2 = int(avg_x2 + ( y2 - avg_y2) / slope)
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
    except ValueError:
        pass
    
def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    left_coords = { 'x1' : [], 'y1' : [], 'x2' : [], 'y2' : [] }
    right_coords = { 'x1' : [], 'y1' : [], 'x2' : [], 'y2' : [] }
    for line in lines:
        for x1, y1, x2, y2 in line:            
            if bad_slope( x1, y1, x2, y2 ):
                continue
            if is_left( x1, y1, x2, y2 ):
                append_coordinates( x1, y1, x2, y2, left_coords )
            else:
                append_coordinates( x1, y1, x2, y2, right_coords )               
                
    render_coordinates( img, left_coords, 320, 540, color, thickness )
    render_coordinates( img, right_coords, 320, 540, color, thickness )

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [643]:
import os 

#this method copied from here: http://flippidybits.com/project-1-lane-finding/
def roi_poly( img ) :
    height, width = img.shape[:2]    
    ratio = 5/8
    roi_left = np.array([[
        (100, height),
        ((1 - ratio) * width, ratio * height),
        (.5 * width, ratio * height),
        (.5 * width, height)
    ]], dtype=np.int32)

    roi_right = np.array([[
        (.5 * width, height),
        (.5 * width, ratio * height),
        (ratio * width, ratio * height),
        (width, height)
    ]], dtype=np.int32)
    
    return ( roi_left, roi_right )


# used by both images (here) and videos (next section).
def process_image(image):

    color_select = np.copy(image)
    
    rgb_threshold = [150, 150, 150]
    thresholds = (color_select[:,:,0] < rgb_threshold[0]) \
            & (color_select[:,:,1] < rgb_threshold[1]) \
            & (color_select[:,:,2] < rgb_threshold[2])
    color_select[thresholds] = [0,0,0]
    
    gray = grayscale( color_select )
    
    # Define a kernel size and apply Gaussian smoothing
    kernel_size = 5
    g_gray = gaussian_blur( gray, kernel_size )

    canny_lt = 50
    canny_ht = 150
    edges = canny( g_gray, canny_lt, canny_ht )

    vertices = roi_poly( edges ) #focus on relevant edges in the image
    masked_edges = region_of_interest( edges, vertices )
    
    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 2
    theta = np.pi/180
    threshold = 20
    min_line_length = 25
    max_line_gap = 10
    hlines = hough_lines( masked_edges, rho, theta, threshold, min_line_length, max_line_gap )

    return weighted_img(hlines, image)

In [644]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.
images = os.listdir("test_images/")
outdir = "test_images_proc"
if not os.path.exists(outdir):
    os.makedirs(outdir)
    
for file in images:    
    # skip files starting with processed
    image = mpimg.imread( 'test_images/' + file )       
    pimage = process_image(image)

    #plt.imshow(pimage)
    mpimg.imsave( outdir + '/' + file, pimage )

In [645]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [646]:
import imageio
imageio.plugins.ffmpeg.download()

video_outdir = "test_videos_proc"
if not os.path.exists(video_outdir):
    os.makedirs(video_outdir)

Let's try the one with the solid white lane on the right first ...

In [647]:
white_output = video_outdir + '/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_proc/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_proc/solidWhiteRight.mp4



100%|█████████▉| 221/222 [00:05<00:00, 40.14it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_proc/solidWhiteRight.mp4 

CPU times: user 5.59 s, sys: 250 ms, total: 5.84 s
Wall time: 6.09 s


In [648]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [649]:
yellow_output = video_outdir + '/solidYellowLeft.mp4'
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_proc/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_proc/solidYellowLeft.mp4



100%|█████████▉| 681/682 [00:18<00:00, 37.68it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_proc/solidYellowLeft.mp4 

CPU times: user 17.8 s, sys: 730 ms, total: 18.5 s
Wall time: 18.7 s


In [650]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

# Reflections

I mostly followed the steps as in tutorial to zero-out the undesired lines and retain only lane segments. Tried tweaking filter params to filter out fragments that don't belong to lane but rather adjoining artifcats ( vehicles etc ). Later realized its easier to exclude them when extrapolating points to render a complete line across the lane fragments.

Main effort was to average endpoints of relavant lines on left and right and render the lines. Took me a while to figure out way to filter lines with bad slopes. Also went through few blogs to learn way to select the most relevant cross sections of the image.

Pipeline will obviously not work well for realtime conditions where there are steeper curves and more noise due to light/shadow/trees etc. Look forward to learn what other techniques are used get around such noise.

## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [651]:
challenge_output = video_outdir + '/challenge.mp4'
clip2 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video test_videos_proc/challenge.mp4
[MoviePy] Writing video test_videos_proc/challenge.mp4



100%|██████████| 251/251 [00:12<00:00, 18.28it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_proc/challenge.mp4 

CPU times: user 10.9 s, sys: 460 ms, total: 11.3 s
Wall time: 13.7 s


In [652]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))